## Loading the environment

In [3]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# MODEL = "gpt-3.5-turbo"
MODEL = "llama3.1"

# Sam Altman: OpenAI CEO on GPT-4, ChatGPT, and the Future of AI
YOUTUBE_VIDEO_URL = "https://www.youtube.com/watch?v=L_Guz73e6fw"


## Setting up llm/slm model

In [4]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

if MODEL.startswith("gpt"):
    model = ChatOpenAI(api_key=OPENAI_API_KEY,model=MODEL)
    embeddings = OpenAIEmbeddings()
else:
    model = Ollama(model=MODEL)
    embeddings = OllamaEmbeddings(model=MODEL)

Simple model test to verify if it's working.

Extracting the response by chaining the model with an output parser

In [6]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
chain = model | parser
chain.invoke("Tell me a joke")

"Here's one:\n\nWhat do you call a fake noodle?\n\nAn impasta!\n\nHope that made you laugh! Do you want to hear another one?"

## Setting up the prompt templates

In [7]:
from langchain.prompts import ChatPromptTemplate

template = """
Answer the question based on the context below.
If you can't answer the question, reply "I don't know"

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))

Human: 
Answer the question based on the context below.
If you can't answer the question, reply "I don't know"

Context: Here is some context

Question: Here is a question



Simple test to see if the model & prompt template was done correctly

In [8]:
chain = prompt | model | parser
chain.invoke({
    "context": "The name I was given was Daniel",
    "question": "What's my name ?"
})

'Daniel.'

## Alternate translation prompt

In [13]:
from operator import itemgetter

translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Daniel want to order calamari for lunch. He does'nt like anything else.",
        "question": "How many dishes did Daniel ordered?",
        "language": "français",
    }
)

'Il a commandé 1 plat (calamari). \n\nOr, cela traduit mieux :\n\n" Il a commandé une portion de calamars".'

## Transcribing YouTube Video

Transcribing a ~2 and half hour video, the following code might take a few minutes.

Note : Using pytubefix instead of pytube. The developers of this library are trying to fix the HTTP 400 response error as of August 21st 2024...

In [22]:
import tempfile
from pytubefix import YouTube
import whisper


if not os.path.exists("ytb_transcript.txt"):
    youtube = YouTube(YOUTUBE_VIDEO_URL)
    audio = youtube.streams.filter(only_audio=True).first()

    # "base" model used for ok accuracy and fast transcribing
    whisper_model = whisper.load_model("base")

    with tempfile.TemporaryDirectory() as tmpdir:
        file = audio.download(output_path=tmpdir)
        transcription = whisper_model.transcribe(file, fp16=False)["text"].strip()

        with open("ytb_transcript.txt", "w") as file:
            file.write(transcription)

100%|███████████████████████████████████████| 139M/139M [00:13<00:00, 11.1MiB/s]
/Users/danielgiaoo/Developer/GitHub/rag-youtube-qna-app/.venv/lib/python3.12/site-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an i